[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/faiss-ebook/faiss-tutorial/intro.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/search/faiss-ebook/faiss-tutorial/intro.ipynb)

In [1]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

# Dataset Initialization

In [2]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

text = res.text
text[:100]

'pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment\n1\tA group of kids is playing in '

We need this in a dataframe, which we build from the `text` string like so:

In [3]:
data = pd.read_csv(StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


We will take all samples from `sentence_A` and build sentence embeddings for each - which we can then store in FAISS.

In [4]:
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

And we will pull in the `sentence_B` column too, giving us ~4.5K *unique* sentences.

In [5]:
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)
len(set(sentences))

4802

This isn't a particularly large number, so let's pull in a few more similar datasets.

In [6]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [7]:
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

b'Skipping line 191: expected 3 fields, saw 4\nSkipping line 206: expected 3 fields, saw 4\nSkipping line 295: expected 3 fields, saw 4\nSkipping line 695: expected 3 fields, saw 4\nSkipping line 699: expected 3 fields, saw 4\n'
b'Skipping line 104: expected 3 fields, saw 4\nSkipping line 181: expected 3 fields, saw 4\nSkipping line 317: expected 3 fields, saw 4\nSkipping line 412: expected 3 fields, saw 5\nSkipping line 508: expected 3 fields, saw 4\n'


In [8]:
len(set(sentences))

14505

Before converting to our sentence embeddings, we will save to text file as backup.

In [9]:
# remove duplicates and NaN
sentences = [
    sentence.replace('\n', '') for sentence in list(set(sentences)) if type(sentence) is str
    ]

In [10]:
with open('sentences.txt', 'w') as fp:
    fp.write('\n'.join(sentences))

Now we have 14.5K *unique* sentences, a much better size. We'll go ahead and build the sentence embeddings (this can take some time, feel free to download the embeddings from [here]()). TODO add link

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(14504, 768)

We can save/load from file in the case of needing to reload the notebook for any reason later.

In [28]:
sentence_embeddings.shape[0]

14504

In [31]:
with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
    np.save(fp, sentence_embeddings[0:256])

In [32]:
# saving data
split = 256
file_count = 0
for i in range(0, sentence_embeddings.shape[0], split):
    end = i + split
    if end > sentence_embeddings.shape[0] + 1:
        end = sentence_embeddings.shape[0] + 1
    file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
    with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
        np.save(fp, sentence_embeddings[i:end, :])
    print(f"embeddings_{file_count}.npy | {i} -> {end}")
    file_count = int(file_count) + 1

embeddings_0.npy | 0 -> 256
embeddings_1.npy | 256 -> 512
embeddings_2.npy | 512 -> 768
embeddings_3.npy | 768 -> 1024
embeddings_4.npy | 1024 -> 1280
embeddings_5.npy | 1280 -> 1536
embeddings_6.npy | 1536 -> 1792
embeddings_7.npy | 1792 -> 2048
embeddings_8.npy | 2048 -> 2304
embeddings_9.npy | 2304 -> 2560
embeddings_10.npy | 2560 -> 2816
embeddings_11.npy | 2816 -> 3072
embeddings_12.npy | 3072 -> 3328
embeddings_13.npy | 3328 -> 3584
embeddings_14.npy | 3584 -> 3840
embeddings_15.npy | 3840 -> 4096
embeddings_16.npy | 4096 -> 4352
embeddings_17.npy | 4352 -> 4608
embeddings_18.npy | 4608 -> 4864
embeddings_19.npy | 4864 -> 5120
embeddings_20.npy | 5120 -> 5376
embeddings_21.npy | 5376 -> 5632
embeddings_22.npy | 5632 -> 5888
embeddings_23.npy | 5888 -> 6144
embeddings_24.npy | 6144 -> 6400
embeddings_25.npy | 6400 -> 6656
embeddings_26.npy | 6656 -> 6912
embeddings_27.npy | 6912 -> 7168
embeddings_28.npy | 7168 -> 7424
embeddings_29.npy | 7424 -> 7680
embeddings_30.npy | 7680 -> 7

We setup our FAISS database dimensionality (number of dimensions per vector) based on these vectors.

In [16]:
d = sentence_embeddings.shape[1]
d

768

# Flat L2 Index

We initialize the flat L2 distance index `IndexFlatL2`, all we need is the specify the vector dimensionality - which in this case is `d == 768` (to align with the sentence-BERT model output embeddings of size `768`).

In [17]:
index = faiss.IndexFlatL2(d)

Often, we will use indexes that require us to `train` them on our data before being used (if we are grouping or transforming the data in any way). `IndexFlatL2` however, is a simple operation and only requires that we calculate distances between vectors when we introduce our query vector `xq` during search. So, in this case, no training is required - which we can confirm by checking the `is_trained` attribute.

In [18]:
index.is_trained

True

Okay so once we're happy that our index is prepared, we then add new vectors using the `add` method.

In [19]:
index.add(sentence_embeddings)

In [20]:
index.ntotal

14504

Then search given a query `xq` and number of nearest neigbors to return `k`.

In [21]:
k = 4
xq = model.encode(["Someone sprints with a football"])

In [22]:
%%time
D, I = index.search(xq, k)  # search
print(I)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36245  6495 57489  8705

[[ 4586 10252 12465   190]]
CPU times: user 16.2 ms, sys: 7.57 ms, total: 23.8 ms
Wall time: 9.05 ms


Here we're returning indices `7460`, `10940`, `3781`, and `5747`, which returns:

In [23]:
[f'{i}: {sentences[i]}' for i in I[0]]

['4586: A group of football players is running in the field',
 '10252: A group of people playing football is running in the field',
 '12465: Two groups of people are playing football',
 '190: A person playing football is running past an official carrying a football']

In [32]:
sentences[7460]

'A group of football players is running in the field'

Clearly we have some good matches, everything returned includes people running with a football, or on the context of a football match. Now, if we'd rather extract the numerical vectors from FAISS, we can do that too.

In [33]:
vecs = np.zeros((k, d))
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [34]:
vecs.shape

(4, 768)

In [35]:
vecs[0][:100]

array([ 0.01627046,  0.22325929, -0.15037425, -0.30747262, -0.27122435,
       -0.10593167, -0.0646093 ,  0.04738174, -0.73349041, -0.37657705,
       -0.76762843,  0.16902871,  0.53107643,  0.51176691,  1.14415848,
       -0.08562929, -0.67240077, -0.96637076,  0.02545463, -0.21559823,
       -1.25656605, -0.82982153, -0.09825023, -0.21850856,  0.50610232,
        0.10527933,  0.50396878,  0.65242976, -1.39458692,  0.6584745 ,
       -0.21525341, -0.22487433,  0.81818348,  0.084643  , -0.76141697,
       -0.28928292, -0.09825806, -0.73046201,  0.07855812, -0.84354657,
       -0.59242058,  0.77471375, -1.20920527, -0.22757971, -1.30733597,
       -0.23081468, -1.31322539,  0.01629073, -0.97285444,  0.19308169,
        0.47424558,  1.18920875, -1.96741307, -0.70061135, -0.29638717,
        0.60533702,  0.62407452, -0.70340395, -0.86754197,  0.17673187,
       -0.1917053 , -0.02951987,  0.22623563, -0.16695444, -0.80402541,
       -0.45918915,  0.69675523, -0.24928184, -1.01478696, -0.92

---

# Adding Partitioning to the Index

FAISS allows us to add an additional step to optimize our search efficiency using a variety of different methods. A popular approach is to partition the index into *[Voronoi cells](https://en.wikipedia.org/wiki/Voronoi_diagram)*. Using this method we would take our query vector `xq`, identify the *cell* it belongs to, and then use our `IndexFlatL2` to search between the query vector `xq` and all indexed vectors belonging to that *cell*. We can also include vectors from other nearby cells too.

We initialize our new partitioned index by first adding our previous `IndexFlatL2` operation as a quantization step (another step in the search process), and feeding this into the new `IndexIVFFlat` operation like so:

In [36]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

Here we've added a new parameter `nlist`. We use `nlist` to define how many partitions we'd like our index to have. 

When we built the previous, `IndexFlatL2`-only index, we noted that no training was required as no grouping/transformation was required to build that index. Now that we've added partitioning using `IndexIVFFlat`, this is no longer the case. Let's take a look at the `is_trained` attribute.

In [37]:
index.is_trained

False

So, what we need to do now is `train` our index on our data, which we do *before* adding any data to the index (otherwise the index cannot know how to group each vector).

In [38]:
index.train(sentence_embeddings)
index.is_trained

True

Now our index is trained, we add our data just as we did before.

In [39]:
index.add(sentence_embeddings)
index.ntotal

14504

Let's search again using the same indexed sentence embeddings and the same query `xq`.

In [40]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 7460 10940  3781  5747]]
CPU times: user 3.83 ms, sys: 3.25 ms, total: 7.08 ms
Wall time: 2.15 ms


We can increase the number of nearby cells to search too with `nprobe`.

In [41]:
index.nprobe = 10

In [42]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 7460 10940  3781  5747]]
CPU times: user 5.29 ms, sys: 2.7 ms, total: 7.99 ms
Wall time: 1.54 ms


Increasing the number of `nprobe` will improve the accuracy of our search, but cost time. Our earlier `IndexFlatL2`-only search was *exhaustive* (it compared every single vector) and so it identified the closest matches with a perfect accuracy. The smaller our `nprobe` value, the smaller scope that we search. We received perfect results (that matched our previous `IndexFlatL2`-only results - `7460`, `10940`, `3781`, `5747`), however, if we found that we were not getting closely matching results, we could simply bump `nprobe` up further - improving accuracy, but increasing time-taken too.

It's worth noting that the time taken can change with each run too, if we rerun the above block, we usually get a different time:

In [43]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 7460 10940  3781  5747]]
CPU times: user 4.88 ms, sys: 2.62 ms, total: 7.5 ms
Wall time: 1.32 ms


For IVF (and IMI) indexes, before attempting to use the `reconstruct` method, we need to call the `make_direct_map` method - otherwise we will return a `RunetimeError`.

In [45]:
index.reconstruct(7460)

RuntimeError: Error in DirectMap::idx_t faiss::DirectMap::get(faiss::DirectMap::idx_t) const at /Users/distiller/miniconda/conda-bld/faiss-pkg_1623024446684/work/faiss/invlists/DirectMap.cpp:81: direct map not initialized

With `make_direct_map`:

In [47]:
index.make_direct_map()

In [48]:
index.reconstruct(7460)[:100]

array([ 0.01627046,  0.22325929, -0.15037425, -0.30747262, -0.27122435,
       -0.10593167, -0.0646093 ,  0.04738174, -0.7334904 , -0.37657705,
       -0.76762843,  0.16902871,  0.53107643,  0.5117669 ,  1.1441585 ,
       -0.08562929, -0.6724008 , -0.96637076,  0.02545463, -0.21559823,
       -1.256566  , -0.8298215 , -0.09825023, -0.21850856,  0.5061023 ,
        0.10527933,  0.5039688 ,  0.65242976, -1.3945869 ,  0.6584745 ,
       -0.21525341, -0.22487433,  0.8181835 ,  0.084643  , -0.761417  ,
       -0.28928292, -0.09825806, -0.730462  ,  0.07855812, -0.84354657,
       -0.5924206 ,  0.77471375, -1.2092053 , -0.22757971, -1.307336  ,
       -0.23081468, -1.3132254 ,  0.01629073, -0.97285444,  0.19308169,
        0.47424558,  1.1892087 , -1.9674131 , -0.70061135, -0.29638717,
        0.605337  ,  0.6240745 , -0.70340395, -0.86754197,  0.17673187,
       -0.1917053 , -0.02951987,  0.22623563, -0.16695444, -0.8040254 ,
       -0.45918915,  0.69675523, -0.24928184, -1.014787  , -0.92

We've now significantly reduced the search time, what can we do next?

# Quantization

Well, when storing these vectors we're storing the full (eg `Flat`) vector. Now in very big datasets this can quickly become a problem. Typically, we look at big datasets, and when working with large dataset we will find that storing the full vectors consumes too much space.

Fortunately, FAISS comes with the ability to *compress* our vectors using transformations based on *Product Quantization* (PQ). But, what is PQ? Well, we can view it as an additional approximation step similar to our use of **IVF**, which allowed us to *approximate* by reducing the scope of our search. PQ is slightly different however, and approximates the distance (or similarity) calculation instead.

[PQ explanation TODO REMOVE](https://mccormickml.com/2017/10/13/product-quantizer-tutorial-part-1/)

PQ achieves this approximated distance operation by compressing the vectors themselves. This consists of a few steps.

1. We split the every original vector into several subvectors.

2. For each set of subvectors, we perform a clustering operation, creating many centroids for each subvector set.

3. In our vector of subvectors, we replace each subvector with the ID of it's nearest centroid.

In [49]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

Again, we'll need to `train` the index.

In [50]:
index.is_trained

False

In [51]:
index.train(sentence_embeddings)

And `add` our vectors.

In [52]:
index.add(sentence_embeddings)

Let's compare it to our previous index *without* PQ, and an `nprobe` value of `10`.

In [53]:
index.nprobe = 10

In [54]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 5013 10940  7460  5370]]
CPU times: user 3.04 ms, sys: 2.18 ms, total: 5.22 ms
Wall time: 1.33 ms


Through adding PQ we've reduced our search time from ~7.5ms to ~5ms, a small difference on a dataset of this size, but when scaled to larger datasets this can make a huge difference.

Now, we should also notice the slightly different results being returned. Beforehand with our exhaustive L2 search we were returning `7460`, `10940`, `3781`, and `5747`. Now, we see a slightly different order to our results - and two different vectors, `5013` and `5370`.

Each of our speed optimization operations, `IVF` and `PQ`, come at the cost of accuracy. Now, if we print out these results we will nonetheless find that each item is still a relevant match:

In [61]:
[f'{i}: {sentences[i]}' for i in I[0]]

['5013: A group of football players running down the field.',
 '10940: A group of people playing football is running in the field',
 '7460: A group of football players is running in the field',
 '5370: A football player is running past an official carrying a football']

So although we might not get the *perfect* result, we still get close - and thanks to the approximation, we get a significant speed boost